In [1]:
# 1 Importing Dependencies
 
import pandas as pd
import requests
from pymongo import MongoClient
import sys
sys.path.append('/Users/tokar/UNC_DA/MY_WORK/Personal_Code')
from my_api_config import nba_rapidapi_key
import time

In [2]:
# 2 API Calling in Teams, creating "teams" collection

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
collection = db["teams"]

# Dropping collection if already exists
if "teams" in db.list_collection_names():
    db.drop_collection("teams")
    print("Existing 'teams' collection dropped.")

# Setting up API call structure for NBA-API.com 
url = "https://api-nba-v1.p.rapidapi.com/teams"

headers = {
	"X-RapidAPI-Key": nba_rapidapi_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

# Calling api
response = requests.get(url, headers=headers)
response_data = response.json()

# Looping through API response and creating a document for each team
for team in response_data['response']:
    if team['nbaFranchise'] and team['allStar']==False:
        team_document = {"team": team}
        collection.insert_one(team_document)
        print(f"Inserted {team} into MongoDB")
        
# Close the MongoDB connection
mongo.close()

Inserted {'id': 1, 'name': 'Atlanta Hawks', 'nickname': 'Hawks', 'code': 'ATL', 'city': 'Atlanta', 'logo': 'https://upload.wikimedia.org/wikipedia/fr/e/ee/Hawks_2016.png', 'allStar': False, 'nbaFranchise': True, 'leagues': {'standard': {'conference': 'East', 'division': 'Southeast'}, 'vegas': {'conference': 'summer', 'division': None}, 'utah': {'conference': 'East', 'division': 'Southeast'}, 'sacramento': {'conference': 'East', 'division': 'Southeast'}}} into MongoDB
Inserted {'id': 2, 'name': 'Boston Celtics', 'nickname': 'Celtics', 'code': 'BOS', 'city': 'Boston', 'logo': 'https://upload.wikimedia.org/wikipedia/fr/thumb/6/65/Celtics_de_Boston_logo.svg/1024px-Celtics_de_Boston_logo.svg.png', 'allStar': False, 'nbaFranchise': True, 'leagues': {'standard': {'conference': 'East', 'division': 'Atlantic'}, 'vegas': {'conference': 'summer', 'division': None}, 'utah': {'conference': 'East', 'division': 'Atlantic'}, 'sacramento': {'conference': 'East', 'division': 'Atlantic'}}} into MongoDB
I

In [3]:
# 3 API Calling in 2021-2023 Games, creating "games" collection

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
collection = db["games"]

# Dropping collection if already exists
if "games" in db.list_collection_names():
    db.drop_collection("games")
    print("Existing 'games' collection dropped.")

# Setting up API call structure for NBA-API.com 
url = "https://api-nba-v1.p.rapidapi.com/games"

headers = {
	"X-RapidAPI-Key": nba_rapidapi_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

# Creating list for seasons to loop through
season_list = ["2021", "2022", "2023"]

# Calling API looping through seasons
for season in season_list:
    querystring = {"season": season}
    response = requests.get(url, headers=headers, params=querystring)

    # Splitting out each game from the response and putting into MongoDB collection "games"
    if response.status_code == 200:
        response_data = response.json()
        for game in response_data['response']:
            game_document = {"season": season, "game": game}
            collection.insert_one(game_document)
            print(f"Inserted game from season {season} into MongoDB")
    else:
        print(f"Failed to fetch data for season {season}, status code: {response.status_code}")
        
# Close the MongoDB connection
mongo.close()

Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 into MongoDB
Inserted game from season 2021 i

In [4]:
# 4 Code to get list of game IDs from all games 

# Establishing MongoDB Connection, to 'nba_data' database and 'games' collection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
collection = db["games"]

# Query to fetch all documents
query = {}

# Projecttion to include on the game ids and not the "_id" from the documents
projection = {'game.id':1, '_id': 0}

# Execute the query and get a cursor for the results
cursor = collection.find(query, projection)

# Extract the 'game.id' values from each document in the cursor
game_ids= [doc['game']['id'] for doc in cursor]

# Convert the list of game IDs to a DataFrame
df = pd.DataFrame(game_ids, columns=['game_ids'])

# Convert the DataFrame column to a list
game_id_list = df['game_ids'].tolist()

# Close the MongoDB connection
mongo.close()

# Printing list and count of games in list
print(f"{len(game_id_list)} games in game_id_list")
print(game_id_list)

4362 games in game_id_list
[8787, 8788, 8789, 8790, 8791, 8792, 8793, 8846, 8847, 8848, 8849, 8850, 8851, 8893, 8894, 8895, 8896, 8897, 8898, 8899, 8900, 8901, 8902, 8903, 8904, 8905, 8906, 8907, 8908, 8909, 8910, 8911, 8912, 8913, 8914, 8915, 8916, 8917, 8918, 8919, 8920, 8921, 8922, 8923, 8924, 8925, 8926, 8927, 8928, 8929, 8930, 8931, 8932, 8933, 8934, 8935, 8936, 8937, 8938, 8939, 9094, 9095, 9096, 9097, 9098, 9099, 9100, 9101, 9102, 9103, 9104, 9105, 9106, 9107, 9108, 9109, 9110, 9111, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9120, 9121, 9122, 9123, 9124, 9125, 9126, 9127, 9128, 9129, 9130, 9131, 9132, 9133, 9134, 9135, 9136, 9137, 9138, 9139, 9140, 9141, 9142, 9143, 9144, 9145, 9146, 9147, 9148, 9149, 9150, 9151, 9152, 9153, 9154, 9155, 9156, 9157, 9158, 9159, 9160, 9163, 9164, 9165, 9166, 9167, 9168, 9169, 9170, 9171, 9172, 9173, 9174, 9175, 9176, 9177, 9178, 9179, 9180, 9181, 9182, 9183, 9184, 9185, 9186, 9187, 9188, 9189, 9190, 9191, 9192, 9193, 9194, 9195, 9196, 9197, 

In [6]:
# 5 Calling API to game stats from list of game_ids, creating "game_stats" collection

# Establishing MongoDB Connection, to 'nba_data' database and 'games_stats' collection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
collection = db["game_stats"]

# Dropping collection if already exists
if "game_stats" in db.list_collection_names():
    db.drop_collection("game_stats")
    print("Existing 'game_stats' collection dropped.")

# Setting up API call structure for NBA-API.com 
url = "https://api-nba-v1.p.rapidapi.com/games/statistics"

headers = {
	"X-RapidAPI-Key": "bb6c1e3fe9mshf016f82ca36e8c6p19fd11jsn48c79b55dc12",
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

# Looping through game_id_list to call in game stats and add documents to "game_stats" collection
for game_id in game_id_list:
    querystring = {"id":game_id}
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        response_data = response.json()
        collection.insert_one(response_data)
        print(f"Data for game ID {game_id} inserted.")
    else:
        print(f"Failed to fetch data for game_id {game_id}, status code: {response.status_code}")

Existing 'game_stats' collection dropped.
Data for game ID 8787 inserted.
Data for game ID 8788 inserted.
Data for game ID 8789 inserted.
Data for game ID 8790 inserted.
Data for game ID 8791 inserted.
Data for game ID 8792 inserted.
Data for game ID 8793 inserted.
Data for game ID 8846 inserted.
Data for game ID 8847 inserted.
Data for game ID 8848 inserted.
Data for game ID 8849 inserted.
Data for game ID 8850 inserted.
Data for game ID 8851 inserted.
Data for game ID 8893 inserted.
Data for game ID 8894 inserted.
Data for game ID 8895 inserted.
Data for game ID 8896 inserted.
Data for game ID 8897 inserted.
Data for game ID 8898 inserted.
Data for game ID 8899 inserted.
Data for game ID 8900 inserted.
Data for game ID 8901 inserted.
Data for game ID 8902 inserted.
Data for game ID 8903 inserted.
Data for game ID 8904 inserted.
Data for game ID 8905 inserted.
Data for game ID 8906 inserted.
Data for game ID 8907 inserted.
Data for game ID 8908 inserted.
Data for game ID 8909 inserted

In [7]:
# 6 Code to clean up nba_data 'games' collection, removing fields and flattening structure, creating "games_cleaned" collection

# Establishing MongoDB Connection, to 'nba_data' database
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]

# Drop the existing 'games_cleaned' collection if it exists
if "games_cleaned" in db.list_collection_names():
    db.drop_collection("games_cleaned")
    print("Existing 'games_cleaned' collection dropped.")

# Existing collection name
existing_collection_name = "games"

# New collection name
new_collection_name = "games_cleaned"

# Get the existing collection
existing_collection = db[existing_collection_name]

# New collection for documents with renamed fields
new_collection = db[new_collection_name]

# Iterate over documents in the existing collection
for document in existing_collection.find({}):
    # Extract necessary fields from the existing document
    game_id = document["game"]["id"]
    season = document["game"]["season"]
    date_start = document["game"]["date"]["start"]
    arena = document["game"]["arena"]
    visitor = document["game"]["teams"]["visitors"]
    home = document["game"]["teams"]["home"]
    visitors_points = document["game"]["scores"]["visitors"]["points"]
    home_points = document["game"]["scores"]["home"]["points"]
    
    # Create a new document with the desired structure
    new_document = {
        "game_id": game_id,
        "date_start": date_start,
        "visitor_name": visitor["name"],
        "visitor_nickname": visitor["nickname"],
        "visitor_code": visitor["code"],
        "home_id": home["id"],
        "home_name": home["name"],
        "home_nickname": home["nickname"],
        "home_code": home["code"],
        "visitor_points": visitors_points,
        "home_points": home_points
    }
    
    # Insert the new document into the new collection
    new_collection.insert_one(new_document)

# Close MongoDB connection
mongo.close()

In [8]:
# 7 Code to clean up nba_data 'game_stats" collection, removing fields and flattening structure

# Establishing MongoDB Connection, to 'nba_data' database
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]

# Drop the existing 'game_stats_cleaned' collection if it exists
if "game_stats_cleaned" in db.list_collection_names():
    db.drop_collection("game_stats_cleaned")
    print("Existing 'game_stats_cleaned' collection dropped.")

# Existing collection name
existing_collection_name = "game_stats"

# New collection name
new_collection_name = "game_stats_cleaned"

# Get the existing collection
existing_collection = db[existing_collection_name]

# New collection for documents with renamed fields
new_collection = db[new_collection_name]

for document in existing_collection.find({}):
    # Extract necessary fields from the existing document
    game_id = document["parameters"]["id"]
    
    # Check if the 'response' field is not empty
    if document.get("response"):
        # Check if 'statistics' is present in the first element of 'response'
        if document["response"][0].get("statistics"):
            visitor_stats = document["response"][0]["statistics"][0]
        else:
            visitor_stats = {}
        
        # Check if 'statistics' is present in the second element of 'response'
        if len(document["response"]) > 1 and document["response"][1].get("statistics"):
            home_stats = document["response"][1]["statistics"][0]
        else:
            home_stats = {}
    else:
        # If 'response' is empty, set empty dictionaries for stats
        visitor_stats = {}
        home_stats = {}
        
    new_document = {
        "game_id": int(game_id),
        "visitor_points": visitor_stats.get("points"),
        "visitor_fgm": visitor_stats.get("fgm"),
        "visitor_fga": visitor_stats.get("fga"),
        "visitor_fgp": visitor_stats.get("fgp"),
        "visitor_ftm": visitor_stats.get("ftm"),
        "visitor_fta": visitor_stats.get("fta"),
        "visitor_ftp": visitor_stats.get("ftp"),
        "visitor_tpm": visitor_stats.get("tpm"),
        "visitor_tpa": visitor_stats.get("tpa"),
        "visitor_tpp": visitor_stats.get("tpp"),
        "visitor_offReb": visitor_stats.get("offReb"),
        "visitor_defReb": visitor_stats.get("defReb"),
        "visitor_totReb": visitor_stats.get("totReb"),
        "visitor_assists": visitor_stats.get("assists"),
        "visitor_pFouls": visitor_stats.get("pFouls"),
        "visitor_steals": visitor_stats.get("steals"),
        "visitor_turnovers": visitor_stats.get("turnovers"),
        "visitor_blocks": visitor_stats.get("blocks"),
        "visitor_plusMinus": visitor_stats.get("plusMinus"),
        "home_points": home_stats.get("points"),
        "home_fgm": home_stats.get("fgm"),
        "home_fga": home_stats.get("fga"),
        "home_fgp": home_stats.get("fgp"),
        "home_ftm": home_stats.get("ftm"),
        "home_fta": home_stats.get("fta"),
        "home_ftp": home_stats.get("ftp"),
        "home_tpm": home_stats.get("tpm"),
        "home_tpa": home_stats.get("tpa"),
        "home_tpp": home_stats.get("tpp"),
        "home_offReb": home_stats.get("offReb"),
        "home_defReb": home_stats.get("defReb"),
        "home_totReb": home_stats.get("totReb"),
        "home_assists": home_stats.get("assists"),
        "home_pFouls": home_stats.get("pFouls"),
        "home_steals": home_stats.get("steals"),
        "home_turnovers": home_stats.get("turnovers"),
        "home_blocks": home_stats.get("blocks"),
        "home_plusMinus": home_stats.get("plusMinus")
    }
    
    # Insert the new document into the new collection
    new_collection.insert_one(new_document)

# Close MongoDB connection
mongo.close()


In [9]:
# 8 At some point the API data quality breaks, it seems the source data 'visitor' and 'home' information switches after game_id 10988.  This code switches the values in the columns where game_id is greater than or equal to 10988

# Establishing MongoDB Connection, to 'nba_data' database
client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']
collection = db['game_stats_cleaned']

# Update documents where game_id is greater than or equal to 10988
collection.update_many(
    {"game_id": {"$gte": 10988}},
    [
        {"$set": {
            "visitor_points": "$$CURRENT.home_points",
            "visitor_fgm": "$$CURRENT.home_fgm",
            "visitor_fga": "$$CURRENT.home_fga",
            "visitor_fgp": "$$CURRENT.home_fgp",
            "visitor_ftm": "$$CURRENT.home_ftm",
            "visitor_fta": "$$CURRENT.home_fta",
            "visitor_ftp": "$$CURRENT.home_ftp",
            "visitor_tpm": "$$CURRENT.home_tpm",
            "visitor_tpa": "$$CURRENT.home_tpa",
            "visitor_tpp": "$$CURRENT.home_tpp",
            "visitor_offReb": "$$CURRENT.home_offReb",
            "visitor_defReb": "$$CURRENT.home_defReb",
            "visitor_totReb": "$$CURRENT.home_totReb",
            "visitor_assists": "$$CURRENT.home_assists",
            "visitor_pFouls": "$$CURRENT.home_pFouls",
            "visitor_steals": "$$CURRENT.home_steals",
            "visitor_turnovers": "$$CURRENT.home_turnovers",
            "visitor_blocks": "$$CURRENT.home_blocks",
            "visitor_plusMinus": "$$CURRENT.home_plusMinus",

            "home_points": "$$CURRENT.visitor_points",
            "home_fgm": "$$CURRENT.visitor_fgm",
            "home_fga": "$$CURRENT.visitor_fga",
            "home_fgp": "$$CURRENT.visitor_fgp",
            "home_ftm": "$$CURRENT.visitor_ftm",
            "home_fta": "$$CURRENT.visitor_fta",
            "home_ftp": "$$CURRENT.visitor_ftp",
            "home_tpm": "$$CURRENT.visitor_tpm",
            "home_tpa": "$$CURRENT.visitor_tpa",
            "home_tpp": "$$CURRENT.visitor_tpp",
            "home_offReb": "$$CURRENT.visitor_offReb",
            "home_defReb": "$$CURRENT.visitor_defReb",
            "home_totReb": "$$CURRENT.visitor_totReb",
            "home_assists": "$$CURRENT.visitor_assists",
            "home_pFouls": "$$CURRENT.visitor_pFouls",
            "home_steals": "$$CURRENT.visitor_steals",
            "home_turnovers": "$$CURRENT.visitor_turnovers",
            "home_blocks": "$$CURRENT.visitor_blocks",
            "home_plusMinus": "$$CURRENT.visitor_plusMinus"
        }}
    ]
)

print("Documents updated successfully.")

# Close the MongoDB connection
client.close()


Documents updated successfully.


In [10]:
# 9 Code to join the two cleaned collections by the game id creating a collection called 'joined_games'

client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']

# Drop the existing 'joined_games' collection if it exists
if "joined_games" in db.list_collection_names():
    db.drop_collection("joined_games")
    print("Existing 'joined_games' collection dropped.")

games_collection = db['games_cleaned']
stats_collection = db['game_stats_cleaned']
joined_collection = db['joined_games']

# Fetch documents from games_cleaned
games_cursor = games_collection.find({})
games_df = pd.DataFrame(list(games_cursor))

# Fetch documents from game_stats_cleaned
stats_cursor = stats_collection.find({})
stats_df = pd.DataFrame(list(stats_cursor))

# Perform the join on 'game_id'
merged_df = pd.merge(games_df, stats_df, on='game_id', suffixes=('_game', '_stats'))

merged_df.dropna(how='any', inplace=True)

# Convert the merged DataFrame back to a dictionary format
merged_docs = merged_df.to_dict(orient='records')

# Insert the merged documents into the new collection
joined_collection.insert_many(merged_docs)

# Close the MongoDB connection
client.close()

print(f"Inserted {len(merged_docs)} documents into 'joined_games' collection.")


Inserted 4177 documents into 'joined_games' collection.


In [11]:
# 10 Creating a unique key of date/home_team/visitor

client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']
collection = db['joined_games']

# Update documents to add the new field
collection.update_many(
    {},
    [
        {"$set": {
            "unique_key": {
                "$concat": [
                    {"$substr": ["$date_start", 0, 10]},
                    "$home_name",
                    "$visitor_name"
                ]
            }
        }}
    ]
)

print("Documents updated successfully.")

# Close the MongoDB connection
client.close()

Documents updated successfully.
